# Part 2 - Generalization Across Subjects

### 1. Apply the Preprocessing Pipeline to the Data of all Subjects

### 2. Extract the same set of features and compare them

### 3. Train a classification model on 26 subjects and evaluate on the 27th subject

### 4. Cross Validation: Repeat **Part 3** for each subject

### 5. Experiment with different numbers of subjects in the **Test Set**